In [ ]:
import visual_behavior.data_access.loading as loading
import visual_behavior.data_access.utilities as data_utilities
import visual_behavior.validation.sdk as sdk_validation
from visual_behavior.validation.sdk import ValidateSDK
from visual_behavior import database as db

import datetime
import pandas as pd

%widescreen
%standard_imports

In [ ]:
!which python

In [ ]:
def get_donor_from_specimen_id(specimen_id):
    res = db.lims_query('select * from specimens where id = {}'.format(specimen_id))
    if len(res['donor_id']) == 1:
        return res['donor_id'].iloc[0]
    elif len(res['donor_id']) == 0:
        return None
    elif len(res['donor_id']) > 1:
        print('found more than one donor ID for specimen ID {}'.format(specimen_id))
        return res['donor_id'].iloc[0]

cache = loading.get_visual_behavior_cache()
behavior_session_table = cache.get_behavior_session_table().reset_index()
filtered_ophys_experiment_table = loading.get_filtered_ophys_experiment_table()

# get donor id for experiment_table
filtered_ophys_experiment_table['donor_id'] = filtered_ophys_experiment_table['specimen_id'].map(
    lambda sid: get_donor_from_specimen_id(sid)
)

# get behavior donor dataframe - all mice in behavior table
behavior_donors = pd.DataFrame({'donor_id':behavior_session_table['donor_id'].unique()})
# add a flag identifying which donors have associated ophys sessions
behavior_donors['donor_in_ophys'] = behavior_donors['donor_id'].map(
    lambda did: did in list(filtered_ophys_experiment_table['donor_id'].unique())
)

# merge back in behavior donors to determine which behavior sessions have associated ophys
behavior_session_table = behavior_session_table.merge(
    behavior_donors,
    left_on='donor_id',
    right_on='donor_id',
    how='left',
)

# get project table
project_table = db.lims_query("select id,code from projects")
query = '''SELECT behavior_sessions.id, specimens.project_id FROM specimens
JOIN donors ON specimens.donor_id=donors.id
JOIN behavior_sessions ON donors.id=behavior_sessions.donor_id'''
behavior_id_project_id_map = db.lims_query(query).rename(columns={'id':'behavior_session_id'}).merge(
    project_table,
    left_on='project_id',
    right_on='id',
    how='left',
).drop(columns=['id']).rename(columns={'code':'project_code'}).drop_duplicates('behavior_session_id').set_index('behavior_session_id')

# merge project table with behavior sessions
behavior_session_table = behavior_session_table.merge(
    behavior_id_project_id_map.reset_index(),
    left_on='behavior_session_id',
    right_on='behavior_session_id',
    how='left'
)

# add a boolean for whether or not a session is in the filtered experiment table
def osid_in_filtered_experiments(osid):
    if pd.notnull(osid):
        return osid in filtered_ophys_experiment_table['ophys_session_id'].unique()
    else:
        return True
behavior_session_table['in_filtered_experiments'] = behavior_session_table['ophys_session_id'].apply(osid_in_filtered_experiments)

# add missing session types (I have no idea why some are missing!)
def get_session_type(osid):
    if osid in filtered_ophys_experiment_table['ophys_session_id'].unique().tolist():
        return filtered_ophys_experiment_table.query('ophys_session_id == {}'.format(osid)).iloc[0]['session_type']
    else:
        return None

# drop sessions for mice with no ophys AND sessions that aren't in the filtered list
behavior_session_table = behavior_session_table.set_index('behavior_session_id').query('donor_in_ophys and in_filtered_experiments').copy()
for idx,row in behavior_session_table.iterrows():
    if pd.isnull(row['session_type']):
        behavior_session_table.at[idx, 'session_type'] = get_session_type(row['ophys_session_id'])

validation_results = sdk_validation.get_validation_results().sort_index()
behavior_session_table = behavior_session_table.merge(
    validation_results,
    left_index=True,
    right_index=True,
    how='left'
)

In [ ]:
missing_sessions = behavior_session_table[pd.isnull(behavior_session_table['metadata'])].index.values
len(missing_sessions)

In [ ]:
savefig = True

sort_by = ['is_ophys','project_code','session_type','behavior_session_id']

fig = sdk_validation.make_sdk_heatmap(
    behavior_session_table.sort_values(by=sort_by)
)
if savefig:
    fig.write_html("/home/dougo/code/dougollerenshaw.github.io/figures_to_share/sdk_validation_matrix.html")
fig.show()

In [ ]:
savefig = True

sort_by = ['is_ophys','project_code','session_type','behavior_session_id']

for title,is_ophys in zip(['ophys_only','behavior_only'],[True,False]):
    fig = sdk_validation.make_sdk_heatmap(
        behavior_session_table[
            (behavior_session_table['is_ophys'] == is_ophys)
        ].sort_values(by=sort_by),
        title_addendum = ' - {} - '.format(title)
    )
    if savefig:
        fig.write_html("/home/dougo/code/dougollerenshaw.github.io/figures_to_share/sdk_validation_matrix_{}.html".format(title))
    fig.show()

In [ ]:
import allensdk
allensdk.__version__